# (Optional) 웹 크롤링2 - Dynamic Crawling

# 0. 라이브러리

In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time
#모듈, 패키지 등 불러오기

# 1. Selenium 기초
자신의 크롬 버전을 확인하고 크롬 웹드라이버를 다운받아놓아야합니다.

* 2020.09.13 기준 최신 버전: ```85.0.4183.102```

## 1.1. Simple Text Crawling
멜론 사이트에서 노래 제목을 크롤링해보자

URL: https://www.melon.com/chart/index.htm

In [62]:
DRIVER_PATH = 'C:/Users/user/Downloads/chromedriver_win32/chromedriver.exe' #경로 설정

In [65]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm" #url 할당

driver.get(url) #브라우저가 url 얻음
html = driver.page_source #웹페이지의 소스 가져오기
soup = BeautifulSoup(html, 'html.parser') #객체 생성

# title crawling
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#frm > div > table > tbody > tr:nth-child(1) > td:nth-child(6) > div > div")))
#내가 지정한 요소가 다 뜰 때 까지 driver를 20초 기다림
#6번째로 바뀜
# print("Title: {}".format(title.text))

title.text

'STAY\nThe Kid LAROI, Justin Bieber'

css selector의 규칙을 찾아본다

* 1번째 제목: #frm > div > table > tbody > tr:nth-child(1) > td:nth-child(4) > div > div"
* 2번째 제목: #frm > div > table > tbody > tr:nth-child(2) > td:nth-child(4) > div > div
. . .

* 100번째 제목: #frm > div > table > tbody > tr:nth-child(100) > td:nth-child(4) > div > div
또는 XPATH로도 확인해보자 (full Xpath)

* 1번째 제목: //*[@id="frm"]/div/table/tbody/tr[1]/td[4]/div/div
* 2번째 제목: //*[@id="frm"]/div/table/tbody/tr[2]/td[4]/div/div
. . .

* 50번째 제목: //*[@id="frm"]/div/table/tbody/tr[100]/td[4]/div/div

In [67]:
# 2번째 제목 크롤링
WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.XPATH, "//*[@id='frm']/div/table/tbody/tr[2]/td[6]/div/div"))).text

'My Universe\nColdplay, 방탄소년단'

## 1.2. Text Crawling with for loop
위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자

In [69]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

#url 열기
url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
#1~50까지 반복
#내가 지정한 요소를 title에 추가하고 text를 list 뒤에 추가
#td 4->6번째로 바뀜
for i in range(1, 51):
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"//*[@id='frm']/div/table/tbody/tr[{i}]/td[6]/div/div")))
    title_list.append(title.text)
    
print(title_list)


['STAY\nThe Kid LAROI, Justin Bieber', 'My Universe\nColdplay, 방탄소년단', '신호등\n이무진', 'OHAYO MY NIGHT\n디핵 (D-Hack), PATEKO (파테코)', 'Permission to Dance\n방탄소년단', 'Next Level\naespa', 'Butter\n방탄소년단', '바라만 본다\nMSG워너비(M.O.M)', 'Weekend\n태연 (TAEYEON)', '낙하 (with 아이유)\nAKMU (악뮤)', '이제 나만 믿어요\n임영웅', 'Dynamite\n방탄소년단', 'Queendom\nRed Velvet (레드벨벳)', '시간을 거슬러 (낮에 뜨는 달 X 케이윌)\n케이윌', '다정히 내 이름을 부르면\n경서예지, 전건호', 'Sticker\nNCT 127', 'DUMB DUMB\n전소미', '좋아좋아\n조정석', 'Peaches (Feat. Daniel Caesar & Giveon)\nJustin Bieber', 'Bad Habits\nEd Sheeran', '가을 타나 봐\n이무진', '별빛 같은 나의 사랑아\n임영웅', '헤픈 우연\n헤이즈 (Heize)', 'HERO\n임영웅', '그대라는 사치\n임영웅', '다시 사랑한다면 (김필 Ver.)\n임영웅', 'Lemonade\nNCT 127', '끝사랑\n임영웅', '고백\n멜로망스', '고백\n어반자카파', 'Bk Love\n임영웅', '잊었니\n임영웅', '비와 당신\n이무진', '찰나가 영원이 될 때 (The Eternal Moment)\n마크툽 (MAKTUB)', 'Savage Love (Laxed - Siren Beat) (BTS Remix)\nJawsh 685, Jason Derulo, 방탄소년단', '색안경 (STEREOTYPE)\nSTAYC(스테이씨)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)\n방탄소년단', '미워요\n임영웅', 'Road Trip\nNCT 127', 

### 1.3. Text Crawling (Click & Back)
클릭하고 나오기 -> 동적 크롤링 가능 (가사 크롤링 가능)

노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 XPath를 추가한다.